In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

laptop=pd.read_csv('Laptop_dlxuly.csv')
laptop.head(5)

,Thương hiệu,Tên sp,Giá,Tên khách hàng,Rating,Hài lòng,Đánh giá,RAM,Loại RAM,Tốc độ Bus RAM,...,Kết nối không dây,Webcam,Đèn bàn phím,Chất liệu,Thông tin Pin,Hệ điều hành,Thời điểm ra mắt,Dài(mm),Rộng(mm),Trọng Lượng(kg)
0,Acer,Laptop Acer Aspire 3 A314 42P R3B3 R7 5700U/16...,12490000,Pham Dung,1,1,Máy để chế độ sleep khoảng 12 tiếng sẽ bị nóng...,16 GB,LPDDR4X (Onboard),4266 MHz,...,Wi-Fi 6 (802.11ax),HD webcam,Không có đèn,Vỏ nhựa,"3-cell, 50Wh",Windows 11 Home SL,2024,318.2,225.5,1.4
1,Acer,Laptop Acer Aspire 3 A314 42P R3B3 R7 5700U/16...,12490000,Lê Văn Đối,1,1,Mua sản phẩm vào tháng 09/2024 đến tháng 02/20...,16 GB,LPDDR4X (Onboard),4266 MHz,...,Wi-Fi 6 (802.11ax),HD webcam,Không có đèn,Vỏ nhựa,"3-cell, 50Wh",Windows 11 Home SL,2024,318.2,225.5,1.4
2,Acer,Laptop Acer Aspire 3 A314 42P R3B3 R7 5700U/16...,12490000,Long Hoang,5,1,trong tâm gia nay thi ryzen 7 5800u đap ưng qu...,16 GB,LPDDR4X (Onboard),4266 MHz,...,Wi-Fi 6 (802.11ax),HD webcam,Không có đèn,Vỏ nhựa,"3-cell, 50Wh",Windows 11 Home SL,2024,318.2,225.5,1.4
3,Acer,Laptop Acer Aspire 3 A314 42P R3B3 R7 5700U/16...,12490000,Hải Đăng,3,1,không có gì nổi trội hết,16 GB,LPDDR4X (Onboard),4266 MHz,...,Wi-Fi 6 (802.11ax),HD webcam,Không có đèn,Vỏ nhựa,"3-cell, 50Wh",Windows 11 Home SL,2024,318.2,225.5,1.4
4,Acer,Laptop Acer Aspire 3 A314 42P R3B3 R7 5700U/16...,12490000,Nguyễn Huy Tuấn,4,1,Mới mua con này hôm 23/9 mà hôm sau giá giảm t...,16 GB,LPDDR4X (Onboard),4266 MHz,...,Wi-Fi 6 (802.11ax),HD webcam,Không có đèn,Vỏ nhựa,"3-cell, 50Wh",Windows 11 Home SL,2024,318.2,225.5,1.4


In [2]:
laptop.columns

Index(['Thương hiệu', 'Tên sp', 'Giá', 'Tên khách hàng', 'Rating', 'Hài lòng',
       'Đánh giá', 'RAM', 'Loại RAM', 'Tốc độ Bus RAM', 'Hỗ trợ RAM tối đa',
       'Ổ cứng', 'Công nghệ CPU', 'Số nhân', 'Số luồng', 'Tốc độ CPU',
       'Tốc độ tối đa', 'Màn hình', 'Độ phân giải', 'Tần số quét',
       'Công nghệ màn hình', 'Card màn hình', 'Công nghệ âm thanh',
       'Cổng giao tiếp', 'Kết nối không dây', 'Webcam', 'Đèn bàn phím',
       'Chất liệu', 'Thông tin Pin', 'Hệ điều hành', 'Thời điểm ra mắt',
       'Dài(mm)', 'Rộng(mm)', 'Trọng Lượng(kg)'],
      dtype='object')

In [3]:
data = laptop[['Thương hiệu', 'Tên sp', 'Giá', 'Đánh giá', 'RAM', 'Loại RAM', 'Ổ cứng', 'Công nghệ CPU', 'Tốc độ CPU',
                 'Màn hình', 'Độ phân giải', 'Card màn hình', 'Chất liệu', 'Thông tin Pin', 'Hệ điều hành']]
data.head(2)

,Thương hiệu,Tên sp,Giá,Đánh giá,RAM,Loại RAM,Ổ cứng,Công nghệ CPU,Tốc độ CPU,Màn hình,Độ phân giải,Card màn hình,Chất liệu,Thông tin Pin,Hệ điều hành
0,Acer,Laptop Acer Aspire 3 A314 42P R3B3 R7 5700U/16...,12490000,Máy để chế độ sleep khoảng 12 tiếng sẽ bị nóng...,16 GB,LPDDR4X (Onboard),"512 GB SSD NVMe PCIe (Có thể tháo ra, lắp than...",AMD Ryzen 7 - 5700U,1.8GHz,"14""",WUXGA,Card tích hợp - AMD Radeon Graphics,Vỏ nhựa,"3-cell, 50Wh",Windows 11 Home SL
1,Acer,Laptop Acer Aspire 3 A314 42P R3B3 R7 5700U/16...,12490000,Mua sản phẩm vào tháng 09/2024 đến tháng 02/20...,16 GB,LPDDR4X (Onboard),"512 GB SSD NVMe PCIe (Có thể tháo ra, lắp than...",AMD Ryzen 7 - 5700U,1.8GHz,"14""",WUXGA,Card tích hợp - AMD Radeon Graphics,Vỏ nhựa,"3-cell, 50Wh",Windows 11 Home SL


In [4]:
def clean(text):
    if pd.isnull(text):
        return ''
    text = text.lower().strip()
    text = re.sub(r'[^\w\s.]', ' ', text, flags=re.UNICODE)
    for unit in ['gb', 'tb', 'mb', 'ghz', 'mhz', 'hz', 'wh', 'inch', 'cell']:
        text = re.sub(rf'(\d+(?:\.\d+)?)\s*{unit}', rf'\1{unit}', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [5]:
columns = ['Thương hiệu', 'Đánh giá', 'RAM', 'Loại RAM', 'Ổ cứng', 'Công nghệ CPU', 'Tốc độ CPU',
            'Màn hình', 'Độ phân giải', 'Card màn hình', 'Chất liệu', 'Thông tin Pin', 'Hệ điều hành']

for col in columns:
    data[col] = data[col].apply(clean)

C:\Users\HP\AppData\Local\Temp\ipykernel_3756\2609018627.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].apply(clean)


In [6]:
data['Tag'] = data[columns].apply(lambda row: ' '.join(row), axis=1)

C:\Users\HP\AppData\Local\Temp\ipykernel_3756\507086554.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Tag'] = data[columns].apply(lambda row: ' '.join(row), axis=1)


In [7]:
data.head(5)

,Thương hiệu,Tên sp,Giá,Đánh giá,RAM,Loại RAM,Ổ cứng,Công nghệ CPU,Tốc độ CPU,Màn hình,Độ phân giải,Card màn hình,Chất liệu,Thông tin Pin,Hệ điều hành,Tag
0,acer,Laptop Acer Aspire 3 A314 42P R3B3 R7 5700U/16...,12490000,máy để chế độ sleep khoảng 12 tiếng sẽ bị nóng...,16gb,lpddr4x onboard,512gb ssd nvme pcie có thể tháo ra lắp thanh k...,amd ryzen 7 5700u,1.8ghz,14,wuxga,card tích hợp amd radeon graphics,vỏ nhựa,3cell 50wh,windows 11 home sl,acer máy để chế độ sleep khoảng 12 tiếng sẽ bị...
1,acer,Laptop Acer Aspire 3 A314 42P R3B3 R7 5700U/16...,12490000,mua sản phẩm vào tháng 09 2024 đến tháng 02 20...,16gb,lpddr4x onboard,512gb ssd nvme pcie có thể tháo ra lắp thanh k...,amd ryzen 7 5700u,1.8ghz,14,wuxga,card tích hợp amd radeon graphics,vỏ nhựa,3cell 50wh,windows 11 home sl,acer mua sản phẩm vào tháng 09 2024 đến tháng ...
2,acer,Laptop Acer Aspire 3 A314 42P R3B3 R7 5700U/16...,12490000,trong tâm gia nay thi ryzen 7 5800u đap ưng qu...,16gb,lpddr4x onboard,512gb ssd nvme pcie có thể tháo ra lắp thanh k...,amd ryzen 7 5700u,1.8ghz,14,wuxga,card tích hợp amd radeon graphics,vỏ nhựa,3cell 50wh,windows 11 home sl,acer trong tâm gia nay thi ryzen 7 5800u đap ư...
3,acer,Laptop Acer Aspire 3 A314 42P R3B3 R7 5700U/16...,12490000,không có gì nổi trội hết,16gb,lpddr4x onboard,512gb ssd nvme pcie có thể tháo ra lắp thanh k...,amd ryzen 7 5700u,1.8ghz,14,wuxga,card tích hợp amd radeon graphics,vỏ nhựa,3cell 50wh,windows 11 home sl,acer không có gì nổi trội hết 16gb lpddr4x onb...
4,acer,Laptop Acer Aspire 3 A314 42P R3B3 R7 5700U/16...,12490000,mới mua con này hôm 23 9 mà hôm sau giá giảm t...,16gb,lpddr4x onboard,512gb ssd nvme pcie có thể tháo ra lắp thanh k...,amd ryzen 7 5700u,1.8ghz,14,wuxga,card tích hợp amd radeon graphics,vỏ nhựa,3cell 50wh,windows 11 home sl,acer mới mua con này hôm 23 9 mà hôm sau giá g...


In [18]:
data['Tag'] = data[columns].apply(lambda row: ' '.join(row), axis=1)
data.iloc[10]['Tag']

C:\Users\HP\AppData\Local\Temp\ipykernel_3756\228477689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Tag'] = data[columns].apply(lambda row: ' '.join(row), axis=1)


'acer rất tốt 8gb ddr4 2 khe 1 khe 8gb 1 khe rời 512gb ssd nvme pcie có thể tháo ra lắp thanh khác tối đa 1tb intel core i5 alder lake 1235u 1.3ghz 15.6 full hd 1920 x 1080 card tích hợp intel uhd graphics iris xe graphics chỉ hoạt động với ram kênh đôi vỏ nhựa 3cell 40wh windows 11 home sl'

In [9]:
data_new = data[['Tên sp', 'Giá', 'Tag']]
data_new.head(2)

,Tên sp,Giá,Tag
0,Laptop Acer Aspire 3 A314 42P R3B3 R7 5700U/16...,12490000,acer máy để chế độ sleep khoảng 12 tiếng sẽ bị...
1,Laptop Acer Aspire 3 A314 42P R3B3 R7 5700U/16...,12490000,acer mua sản phẩm vào tháng 09 2024 đến tháng ...


In [19]:
with open('vietnamese-stopwords.txt', 'r', encoding = 'utf-8') as f:
        vn_stopwords = [line.strip() for line in f if line.strip()]
        vn_stopwords = [word.replace(' ', '_') for word in vn_stopwords]

In [20]:
tfidf = TfidfVectorizer(stop_words = vn_stopwords)
vector = tfidf.fit_transform(data_new['Tag']).toarray()
vector

array([[0.        , 0.        , 0.        , ..., 0.04483891, 0.        ,
        0.        ],
       [0.        , 0.        , 0.1284892 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.15292327, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [12]:
vector.shape

(793, 1774)

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(vector)
similarity_matrix

array([[1.        , 0.14747765, 0.04711437, ..., 0.03878998, 0.03477464,
        0.07024941],
       [0.14747765, 1.        , 0.04964731, ..., 0.09442679, 0.03598281,
        0.12043372],
       [0.04711437, 0.04964731, 1.        , ..., 0.01945393, 0.03488233,
        0.02481191],
       ...,
       [0.03878998, 0.09442679, 0.01945393, ..., 1.        , 0.29729169,
        0.54143819],
       [0.03477464, 0.03598281, 0.03488233, ..., 0.29729169, 1.        ,
        0.37917147],
       [0.07024941, 0.12043372, 0.02481191, ..., 0.54143819, 0.37917147,
        1.        ]])

In [14]:
similarity_matrix.shape

(793, 793)

In [15]:
def query_tag(des):
    t = []
    for col in columns:
        feature = des.get(col, '')
        feature = clean(feature)
        t.append(feature)
    return ' '.join(t)

In [16]:
def recommend(des):
    query = query_tag(des)
    query_vec = tfidf.transform([query]).toarray()
    similarity_scores = cosine_similarity(query_vec, vector)[0]
    product = similarity_scores.argsort()[::-1][1:10]
    print('Các sản phẩm tương tự:')
    for i in product:
        name = data_new.iloc[i]['Tên sp'].title()
        price = data_new.iloc[i]['Giá']
        print(f'{name} - Giá: {price}')

In [17]:
recommend({'Thương hiệu': 'hp',
          'Giá': '20000000',
          'RAM': '16gb'})

Các sản phẩm tương tự:
Laptop Hp 15S Fq5162Tu I5 1235U/8Gb/512Gb/Win11 (7C134Pa) - Giá: 12990000
Laptop Hp 240 G9 I5 1235U/8Gb/512Gb/Win11 (6L1Y2Pa) - Giá: 12990000
Laptop Hp Pavilion 15 Eg2081Tu I5 1240P/16Gb/512Gb/Win11 (7C0Q4Pa) - Giá: 16890000
Laptop Hp Pavilion 15 Eg2081Tu I5 1240P/16Gb/512Gb/Win11 (7C0Q4Pa) - Giá: 16890000
Laptop Hp Pavilion 15 Eg2081Tu I5 1240P/16Gb/512Gb/Win11 (7C0Q4Pa) - Giá: 16890000
Laptop Hp Pavilion 15 Eg3091Tu I7 1355U/16Gb/512Gb/Win11 (8C5L2Pa) - Giá: 20990000
Laptop Hp Pavilion 15 Eg2081Tu I5 1240P/16Gb/512Gb/Win11 (7C0Q4Pa) - Giá: 16890000
Laptop Hp Gaming Victus 15 Fa1139Tx I5 12450H/16Gb/512Gb/4Gb Rtx2050/144Hz/Win11 (8Y6W3Pa) - Giá: 17690000
Laptop Hp Pavilion 15 Eg2081Tu I5 1240P/16Gb/512Gb/Win11 (7C0Q4Pa) - Giá: 16890000
